# Dashboard Liderazgo

In [ ]:
import os

from_drive = True  # same flag you use everywhere

if os.environ.get("ATLAS_BOOTSTRAPPED") != "1":
    # ---------- GIT ON COLAB ONLY ----------
    try:
        from google.colab import userdata

        git_token = userdata.get('gitToken')
        git_user = userdata.get('gitUser')
        git_url = f'https://{git_token}@github.com/rene-aum/Atlas.git'
        branch_to_pull = 'dev'

        os.chdir('/content')

        if not os.path.isdir('Atlas'):
            !git clone {git_url}

        %cd Atlas
        !git fetch origin {branch_to_pull}
        !git checkout {branch_to_pull}
        !git pull origin {branch_to_pull}

        !pip install -r PipelinesConsumo/src/requirements.txt
        %cd PipelinesConsumo

    except Exception as e:
        print(e)
        print('Running in other environment not colab probably!')

    # ---------- DRIVE + SHEETS ----------
    if from_drive:
        from pydrive2.auth import GoogleAuth
        from pydrive2.drive import GoogleDrive
        from google.colab import auth
        from oauth2client.client import GoogleCredentials
        import gspread
        from google.auth import default
        from gspread_dataframe import set_with_dataframe
        import gdown

        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        drive = GoogleDrive(gauth)

        creds, _ = default()
        gc = gspread.authorize(creds)

    os.environ["ATLAS_BOOTSTRAPPED"] = "1"
else:
    print("Bootstrap already done, assuming orchestrator ran it.")

In [ ]:
import sys
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import pytz
# from matplotlib.ticker import FuncFormatter
from datetime import datetime, timedelta
import warnings
import sys
sys.path.append('..')
sys.path.append('../..')
from utils.utils import (get_dates_dataframe,
                       add_year_week,
                       custom_read,
                       process_columns)
from PipelinesConsumo.src.rawAtlas import RawAtlas
from PipelinesConsumo.src.processedAtlas import ProcessedAtlas
from src.transformed import Transformed
from utils.drive_toolbox import(from_drive_to_local,
                             get_last_modification_date_drive,
                             create_sheets_in_drive_folder,
                             update_sheets_in_drive_folder,
                             read_from_google_sheets,
                             list_file_ids_for_drive_folder,
                             create_csv_file_in_drive_folder,
                             write_csv_to_drive,
                             read_csv_from_drive)
from utils.utils import get_dates_dataframe
from src.constants import (atlas_raw_output_folder_id,
                           atlas_consumo_output_folder_id,
                           consumo_sheets_ids_dict,
                           data_source_folder_id,
                           raw_output_ids,
                           folder_id_bauto_gabo,
                           id_reporte_ventas,
                           )


warnings.filterwarnings('ignore')



In [ ]:
consumo_sheets_ids_dict

## Load

In [ ]:
ventas = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcVentas'])
cancelaciones = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcPublicacionesCanceladas'])
publicaciones = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcPublicaciones'])
pedidos = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcPedidos'])
visitas_unicas = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcVisitasUnicas'])
adobe_vendedor_total = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcAdobeFunnelVendedorTotal'])
adobe_comprador_total = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcAdobeFunnelCompradorTotal'])
clientes = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcClientes'])
business_case_df = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcBusinessCase26'])

## Process

In [ ]:
datedf = get_dates_dataframe(start='2024-03-01').assign(date = lambda x: x.date.dt.strftime('%Y-%m-%d'))

In [ ]:
# clientes  
clientes_by_date = (clientes
 .rename(columns={'customer_since':'date'})
 .groupby('date')['id_am'].nunique()
 .to_frame()
 .rename(columns={'id_am':'usuarios_registrados'})
 )

In [ ]:
# visitas unicas
visitas_date = (visitas_unicas
 .groupby(['date']).sum()
 )

In [ ]:
# funnel vendedor
sum_cols_vendedor = adobe_vendedor_total.filter(like='fv_').columns
rename_dict_vendedor = {'fv_app_click_start':'fv_0_app_click_start',
                        'fv_app_page_visit':'fv_1_marca_modelo',
                        'fv_app_step_2':'fv_2_version',
                        'fv_app_step_3':'fv_3_tenemos_tu_version',
                        'fv_app_step_4':'fv_4_precio',
                        'fv_app_step_6':'fv_5_datos_complementarios',
                        'fv_app_step_8':'fv_6_espacio_automarket',
                        'fv_app_step_9':'fv_7_fotografias',
                        'fv_app_step_10':'fv_8_exito_publicacion'
                        }
funnel_vendedor_date = (adobe_vendedor_total
    .groupby(['date'])[sum_cols_vendedor].sum()
    .rename(columns=rename_dict_vendedor)
)

In [ ]:
# funnel comprador
sum_cols_comprador = adobe_comprador_total.filter(like='fc_').columns
rename_dict_comprador = {'fc_app_click_start':'fc_0_app_click_start',
                        'fc_app_page_visit':'fc_1_onboarding',
                        'fc_app_completed':'fc_2_apartado_exitoso',
                        }
funnel_comprador_date = (adobe_comprador_total
    .groupby(['date'])[sum_cols_comprador].sum()
    .rename(columns=rename_dict_comprador)
)

In [ ]:
# cancelaciones
cancelaciones_date = (cancelaciones
 .rename(columns={'cancelled_at':'date'})
 .groupby(['date'])['sku'].nunique()
 .rename('publicaciones_canceladas')
 .to_frame()
 )

In [ ]:
# publicaciones
publicaciones_date = (publicaciones
 [lambda x: x.published_at.notna()]
  .rename(columns={'published_at':'date'})
 .groupby(['date'])['sku'].nunique()
 .rename('publicaciones')
 .to_frame() 
 )
showroom_map = {'Reforma 510':'torre',
                'Metrópoli Patriotismo':'patriotismo',
                'Samara Satélite':'samara'}
publicaciones_espacio_date = (publicaciones
 [lambda x: x.published_at.notna()]
 .assign(year_month=lambda x:pd.to_datetime( x['published_at']).dt.strftime('%Y-%m'),
         showroom = lambda x:x.showroom.map(showroom_map))
  .rename(columns={'published_at':'date'})
 .groupby(['date','showroom'])['sku'].nunique()
 .unstack('showroom')
 .fillna(0)
 )
publicaciones_espacio_date.columns = [f'publicaciones_{showroom}' for showroom in publicaciones_espacio_date.columns]
publicaciones_activas_date = (publicaciones
 [lambda x: x.flag_publicacion_activa==1]
.rename(columns={'published_at':'date'})
 .groupby(['date'])['sku'].nunique()
 .rename('publicaciones_activas')
 .to_frame()
 )

In [ ]:
# pedidos
pedidos_date = (pedidos
 .rename(columns={'fecha_de_creacion':'date'})
 .groupby(['date','multiapartado'])
 ['sf_order_id'].nunique()
 .unstack('multiapartado')
 .rename(columns={0:'apartados_unicos',
               1:'multiapartados'})
 .fillna(0)
 .assign(apartados = lambda x: x.sum(axis=1))
 )

In [ ]:
# ventas
ventas_pago_date = (ventas
 .rename(columns={'fecha_de_entrega':'date'})
 .groupby(['date','tipo_de_venta'])
 .size()
 .rename({0:'ventas'})
 .unstack('tipo_de_venta')
 .fillna(0)
 )
ventas_pago_date.columns = [f'ventas_{tipo}' for tipo in ventas_pago_date.columns]
ventas_pago_date = ventas_pago_date.assign(ventas = lambda x: x.sum(axis=1))
ventas_espacio_date = (ventas
 .rename(columns={'fecha_de_entrega':'date'})
 .groupby(['date','espacio_am'])
 .size()
 .rename({0:'ventas'})
 .unstack('espacio_am')
 .fillna(0)
 )
ventas_espacio_date.columns = [f'ventas_{showroom}' for showroom in ventas_espacio_date.columns]

In [ ]:
# business case 
bc_diario = (business_case_df
    .assign(days_in_month = lambda x: pd.to_datetime(x.mes).dt.days_in_month,
            year_month = lambda x: pd.to_datetime(x.mes).dt.strftime('%Y-%m'))
    
    )
bc_diario = (bc_diario
    .assign(**{c:bc_diario[c]/bc_diario['days_in_month'] for c in bc_diario.columns 
               if c not in ['mes','days_in_month','year_month']})
    .drop(columns=['mes','days_in_month'])
    )

In [ ]:
master = (datedf
          .merge(visitas_date,on='date',how='left')
          .merge(funnel_vendedor_date,on='date',how='left')
          .merge(funnel_comprador_date,on='date',how='left')
          .merge(publicaciones_date,on='date',how='left')
          .merge(publicaciones_activas_date,on='date',how='left')
          .merge(publicaciones_espacio_date,on='date',how='left')
          .merge(cancelaciones_date,on='date',how='left')
          .merge(pedidos_date,on='date',how='left')
          .merge(ventas_espacio_date,on='date',how='left')
          .merge(ventas_pago_date,on='date',how='left')
          .merge(clientes_by_date,on='date',how='left')
          .fillna(0)
          .assign(year = lambda x:pd.to_datetime(x.date).dt.year,
                  month = lambda x:pd.to_datetime(x.date).dt.month,
                  year_month=lambda x:pd.to_datetime( x['date']).dt.strftime('%Y-%m'),
                  year_week = lambda x:pd.to_datetime( x['date']).dt.strftime('%Y-%U'),
                  day_of_week = lambda x:pd.to_datetime( x['date']).dt.strftime('%A'),
                  monday_of_week=lambda x: (pd.to_datetime(x.date) - 
                                            pd.to_timedelta(pd.to_datetime(x.date).dt.dayofweek, unit="d")).dt.strftime("%Y-%m-%d"),
                  )
          .merge(bc_diario,on='year_month',how='left')
          )

## write

In [ ]:
output_master_table_id = '1v8EAje4iFtzSHVoNjQusmjGSjhhnvzaH6vVDlYh0b4U'
update_sheets_in_drive_folder(gc,output_master_table_id,'Hoja 1',master)

## free memory

In [ ]:
vars_to_del = ['ventas','ventas_espacio_date','ventas_pago_date','business_case_df',
               'publicaciones','publicaciones_activas_date','publicaciones_date','publicaciones_espacio_date',
               'pedidos','pedidos_date',
               'cancelaciones_date','cancelaciones',
               'funnel_comprador_date','funnel_vendedor_date',
               'visitas_date',
               'clientes',
               'adobe_comprador_total','adobe_vendedor_total','bc_diario']
for v in vars_to_del:
    try:
        del globals()[v]
    except Exception as e:
        print(f"could not delete var {v}: {e}")

In [ ]:
import gc as gcol
gcol.collect()